In [20]:
import os
import numpy as np

names_train=['dataset/train_folder/'+a for a in os.listdir('dataset/train_folder')]
names_test=['dataset/test_folder/'+a for a in os.listdir('dataset/test_folder')]
names_query=['dataset/query/'+a for a in os.listdir('dataset/query')]
class_names=[] # You should put the class names in this list
LOAD=False
query_idx=1 # It is used for test. If you use a query selected from the test set, you should set this variable to its index


In [2]:
######################################################
### LOAD from already saved files
#####################################################
if LOAD:
    weights_of_blocks_train=np.load('./data/weights_of_blocks_trains_shuffle.npy')
    # Convert all the NaNs into zero. I did not change the original data
    weights_of_blocks_train=np.nan_to_num(weights_of_blocks_train) 
    im_p_test=names_test


    import pandas as pd
    df_test=pd.read_pickle('./data/df_test_features.pkl')

    df_test=pd.read_pickle('./data/df_query_features.pkl')


In [3]:
import sys
import pandas as pd
import numpy as np
from PIL import Image
import time
import cv2
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
# from skimage.metrics import structural_similarity as ssim
from Tools.Color_CDH import color_util
from Tools.Gabor import Gabor_util
from Tools.Blocking import Block_util

import Tools.HNN_weights as HNNWB
import Tools.Calculate_final_weight_HNN as HNNW
from Tools.Block_feat_dataframe import Block_feat_dataframe_LLF

In [4]:
from sklearn.utils import shuffle
im_p_train=shuffle(names_train)


In [5]:
if not(LOAD):
    from datetime import datetime

    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Start Time =", current_time)
    start = time.process_time()

    weights_of_blocks_train,g1err_total,g2err_total,NaNerr_total,df_err=HNNWB.HNN_blocks_weights_calculation_LLF(im_p_train,im_size=256,block_num=4,test_im=0,show_test=0,show_weights=False,save='Oxford5k.jpg')
    print(f"Number of errors in G1 is {g1err_total}  and for G2 is {g2err_total}")
    print(f'Calculating the weight matrix took {time.process_time() - start} seconds')
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("End Time =", current_time)


Start Time = 18:46:49
dataset/train_folder/all_souls_000003.jpg
Calculating Features for Block # 16       Image # 0 is done!
dataset/train_folder/all_souls_000005.jpg
Calculating Features for Block # 16       Image # 1 is done!
dataset/train_folder/all_souls_000001.jpg
Calculating Features for Block # 16       Image # 2 is done!
dataset/train_folder/ashmolean_000003.jpg
Calculating Features for Block # 16       Image # 3 is done!
dataset/train_folder/all_souls_000002.jpg
Calculating Features for Block # 16       Image # 4 is done!
No errors so far! You are an awesome programmer.
The feature set needs 19968 bytes
There are 1408 zeros out of 2496
Calculation of the weight matrix is done for 5 images. Shape of Block_weight is (16, 156, 156)
Number of errors in G1 is 0  and for G2 is 0
Calculating the weight matrix took 21.71875 seconds
End Time = 18:47:10


In [6]:
if not(LOAD):
    from numpy import save
    #Convert all the NaNs to 0.0
    weights_of_blocks_train=np.nan_to_num(weights_of_blocks_train)
    save('./data/weights_of_blocks_trains_shuffle.npy', weights_of_blocks_train)


Up to here, the feature are loaded and stored in weights_of_blocks_train.npy
# Test set calculation


In [7]:
if not(LOAD):
    im_p_test=names_test


    ####################################################################################################################
    # Do you need to recalculate the feature of the test set? if yes, run this cell. otherwise do not run this cell and load the pickle file.
    #####################################################################################################################
    import numpy as np
    import Tools.HNN_weights as HNNWB
    import Tools.Calculate_final_weight_HNN as HNNW
    from Tools.Block_feat_dataframe import Block_feat_dataframe_LLF
    start = time.process_time()
    df_test,feature_shape_test,g1err,g2err,e1,e2=Block_feat_dataframe_LLF(im_p_test,im_size=256,block_num=4,test_im=0,show_test=0)
    print(f'Calculating features for testing set took {time.process_time() - start} seconds')

    #Save df_test containing all the features into a file
    df_test.to_pickle("./data/df_test_features.pkl")

dataset/test_folder/all_souls_000000.jpg
Calculating Features for Block # 16       Image # 0 is done!
dataset/test_folder/all_souls_000001.jpg
Calculating Features for Block # 16       Image # 1 is done!
dataset/test_folder/all_souls_000002.jpg
Calculating Features for Block # 16       Image # 2 is done!
dataset/test_folder/all_souls_000003.jpg
Calculating Features for Block # 16       Image # 3 is done!
dataset/test_folder/all_souls_000005.jpg
Calculating Features for Block # 16       Image # 4 is done!
No errors so far! You are an awesome programmer.
The feature set needs 19968 bytes
There are 1436 zeros out of 2496
Calculating features for testing set took 21.265625 seconds


In [8]:
if not(LOAD):
    start = time.process_time()
    im_p_query=names_query
    df_query,feature_shape_query,_,_,_,_=Block_feat_dataframe_LLF(im_p_query,im_size=256,block_num=4,test_im=0,show_test=0)
    print(f'Calculting features for queries took {time.process_time() - start} seconds')
    df_query.to_pickle("./data/df_query_features.pkl")



dataset/query/all_souls_000000.jpg
Calculating Features for Block # 16       Image # 0 is done!
No errors so far! You are an awesome programmer.
The feature set needs 19968 bytes
There are 1395 zeros out of 2496
Calculting features for queries took 4.53125 seconds


In [9]:
if LOAD:
    df_test=pd.read_pickle('./data/df_test_features.pkl')
    df_query=pd.read_pickle('./data/df_query_features.pkl')

In [10]:
def querying(P_M,query,number_image_to_return=2):

    total_number_of_similar_images=0
    P_M_sorted=P_M.copy(deep=True)
    P_M_sorted=P_M_sorted.sort_values(by="l",ascending=True,ignore_index=True)
    P_M_sorted=P_M_sorted.sort_index()
    for cls in class_names:
        if cls in query.iloc[0,0]:
            Classname=cls

    for i in range(len(P_M_sorted.index)):
        if Classname in P_M_sorted.at[i,'filename']:
            total_number_of_similar_images+=1

    start = int(round(time.time() * 1000000))


    similar=0
    list_of_images=['']
    for i in range(number_image_to_return):
        list_of_images=np.vstack((list_of_images,P_M_sorted.iloc[i,0]))

        if Classname in P_M_sorted.iloc[i,0]:
            similar+=1

    time_took=int(round(time.time() * 1000000)) - start
        
    precision=similar/number_image_to_return
    recall=similar/total_number_of_similar_images
    list_of_images=np.delete(list_of_images,0) # Removing the first empty folder name because of initilizing it with ''

    return precision,recall,time_took,list_of_images

In [16]:
from tqdm import tqdm
start = time.process_time()
step=100
# def cal_Pn(B,feat,W_b):
feat_hat_test=df_test.copy(deep=True)
feat_hat_query=df_query.copy(deep=True)

number_image_to_return=50
B=[0.5,0.5,0.5,0.5,0.5,2,2,0.5,0.5,2,2,0.5,0.5,0.5,0.5,0.5]

numiter=10 # Number of iterations you want for the Hopfield to iterate. It should be >5
feat_shape=np.shape(df_test.iloc[0,1])[0]
num_B=np.shape(B)[0]
num_image=df_test.shape
num_image=num_image[0]
num_image_query=df_query.shape
num_image_query=num_image_query[0]

P_M=pd.DataFrame(columns =['filename','P','s','l']) 
P_M_query=pd.DataFrame(columns =['filename','P','s','l']) 

for i in range(num_image):
    P_M.at[i,"filename"]=df_test.at[i,"filename"]
    P_M.at[i,"P"]=np.zeros([1,feat_shape])
    P_M.at[i,"s"]=0
    P_M.at[i,"l"]=0

for i in range(num_image_query):
    P_M_query.at[i,"filename"]=df_query.at[i,"filename"]
    P_M_query.at[i,"P"]=np.zeros([1,feat_shape])
    P_M_query.at[i,"s"]=0
    P_M_query.at[i,"l"]=0


feat_hat_p=np.zeros([num_image,num_B,feat_shape])
feat_hat_p_query=np.zeros([num_image_query,num_B,feat_shape])

for nIm in range(num_image):
    P_M.at[nIm,'filename']=df_test.iloc[nIm,0]
    for b in range(num_B):
        feat_hat_p[nIm][b][:]=df_test.iloc[nIm][b+1]

for nIm in range(num_image_query):
    P_M_query.at[nIm,'filename']=df_query.iloc[nIm,0]
    for b in range(num_B):
        feat_hat_p_query[nIm][b][:]=df_query.iloc[nIm][b+1]

feat_hat_org=feat_hat_p  # Just to test if the changed feature actually changed?

feat_hat_cal=np.zeros([1,feat_shape])
feat_hat_cal_query=np.zeros([1,feat_shape])
for nIm in range(num_image):
    for b in range(num_B):
        feat_hat_cal=feat_hat_p[nIm][b][:]
        for itr in range(numiter):
            activations=np.matmul(weights_of_blocks_train[b],feat_hat_cal)
        feat_hat_test.iloc[nIm,b+1]=B[b]*activations # I did this to make everything separated as much as possible for further change if required
        P_M.at[nIm,'P']=P_M.at[nIm,'P'] +activations
    P_M.at[nIm,'s']=sum(sum(P_M.at[nIm,'P']))


for nIm in range(num_image_query):
    for b in range(num_B):
        feat_hat_cal_query=feat_hat_p_query[nIm][b][:]
        for itr in range(numiter):
            activations=np.matmul(weights_of_blocks_train[b],feat_hat_cal_query)
        feat_hat_query.iloc[nIm,b+1]=B[b]*activations # I did this to make everything separated as much as possible for further change if required
        P_M_query.at[nIm,'P']=P_M_query.at[nIm,'P'] +activations
    P_M_query.at[nIm,'s']=sum(sum(P_M_query.at[nIm,'P']))

#Start the calculation of |s^i-s^q|
results_dic={}
res_time_dif_num_query=pd.DataFrame(columns =['Number_ret','Time'])
idx=0
for number_image_to_return in tqdm(range(1,num_image+1,step)):
    res=pd.DataFrame(columns =['query','Precision','Recall'])
    for num_q in range(num_image_query):
        res.at[num_q,'query']=P_M_query.at[num_q,'filename']
        for nIm in range(num_image):
            P_M.at[nIm,'l']=abs(P_M.at[nIm,'s']-P_M_query.at[num_q,'s'])

        p,r,t,list_of_images=querying(P_M=P_M,query=P_M_query.iloc[num_q,:].to_frame().transpose(),number_image_to_return=number_image_to_return)
        res.at[num_q,'Precision']=p
        res.at[num_q,'Recall']=r
    results_dic[idx]=res
    finish_t=time.process_time() - start
    res_time_dif_num_query.at[idx,'Number_ret']=number_image_to_return
    res_time_dif_num_query.at[idx,'Time']=finish_t
    idx+=1



100%|██████████| 1/1 [00:00<00:00, 334.26it/s]


In [17]:
from Tools import DCT
from Tools import DWT
from Tools import BST
def hash_cal(imp):
    wSlid_DCT=DCT.windows_comput(imp)
    binCode_DCT=DCT.DCT_hash_compute(wSlid_DCT)
    hashCode_DCT=DCT.bin_dec(binCode_DCT)
    
    wSlid_DWT=DWT.windows_comput(imp,block_num=8)
    binCode_DWT=DWT.DWT_hash_compute(wSlid_DWT)
    hashCode_DWT=DWT.bin_dec(binCode_DWT)
    binCode_DWTp1,binCode_DWTp2=DWT.bin_two_parts(binCode_DWT)
    hashCode_DWTp1=DWT.bin_dec(binCode_DWTp1)
    hashCode_DWTp2=DWT.bin_dec(binCode_DWTp2)
    return hashCode_DCT,hashCode_DWTp1,hashCode_DWTp2,hashCode_DWT
    

In [21]:
import warnings
warnings.filterwarnings('ignore')
df_hash_testset = pd.DataFrame(columns=['index', 'filename', 'DCT','DWT_L','DWT_R','DWT'])
im_path=names_test
idx=0
start = time.process_time()

for imp in tqdm(list_of_images):
    hashCode_DCT,hashCode_DWTp1,hashCode_DWTp2,hashCode_DWT=hash_cal(imp)
    df_hash_testset = df_hash_testset.append({'index': idx, 'filename':imp,'DCT':hashCode_DCT,
                                                'DWT_L':hashCode_DWTp1,'DWT_R':hashCode_DWTp2,'DWT':hashCode_DWT}, ignore_index=True)
    idx += 1
finish_t=time.process_time() - start

# Instead of using query_idx, you should use query datafram df_query if there are more than one images or you are not using one (or more) images ...
# from the test set as queries. In that case, you should do a similar alogirthm as lines 8-12. Then you need to alter below cells a little to match  it.
# I prefer to clean the code by putting one query here. I found the automatic way reduces the readibility of the code.
hashCode_DCT_query,hashCode_DWTp1_query,hashCode_DWTp2_query,hashCode_DWT_query=hash_cal(P_M.iloc[query_idx,0])



100%|██████████| 1/1 [00:00<00:00,  3.19it/s]


In [22]:
def look_in_tree(hashCode_query,BST,list_H,thresh):
        
    lookfor=0
    for i in range(len(list_H)):
        cal=hashCode_query-list_H[i]

        if cal<thresh and cal!=0:
            lookfor=list_H[i+50]
            #print ('Found in index=',i)
            return BST.findsuccessorforN(lookfor).inorder([])

In [23]:
import sys
t1=t2=30
t3=20
for i in tqdm(range(1)):
    start = time.process_time()
    df=df_hash_testset
    df_hash_testset_shuffle = df.sample(frac = 1,ignore_index=True)
    df_hash_testset_shuffle=df_hash_testset_shuffle.sort_index()
    df_hash_testset_shuffle.to_pickle('1.pkl')
    BST_DCT=BST.BSTNode()
    BST_DWT1=BST.BSTNode()
    BST_DWT2=BST.BSTNode()
    BST_DWT=BST.BSTNode()
    for i in range(len(df)):
        BST_DCT.insert(df_hash_testset_shuffle.at[i,'DCT'])
        BST_DWT1.insert(df_hash_testset_shuffle.at[i,'DWT_L'])
        BST_DWT2.insert(df_hash_testset_shuffle.at[i,'DWT_R'])
        BST_DWT.insert(df_hash_testset_shuffle.at[i,'DWT_R'])

    finish_t=time.process_time() - start
    print(f"tree building is done in {finish_t}")

    start = time.process_time()

    DCT_inorder=BST_DCT.inorder([])
    DWTp1_inorder=BST_DWT1.inorder([])
    DWTp2_inorder=BST_DWT2.inorder([])
    DWT_inorder=BST_DWT.inorder([])

    thresh_DCT=hashCode_DCT_query
    thresh_DWTp1=hashCode_DWTp1_query
    thresh_DWTp2=hashCode_DWTp2_query
    thresh_DWT=hashCode_DWT_query
    delta1=look_in_tree(hashCode_DCT_query,BST_DCT,DCT_inorder,thresh_DCT)
    #print(len(delta1))
    delta2=look_in_tree(hashCode_DWTp1_query,BST_DWT1,DWTp1_inorder,thresh_DWTp1)
    #print(len(delta2))
    delta3=look_in_tree(hashCode_DWTp2_query,BST_DWT2,DWTp2_inorder,thresh_DWTp2)
    #print(len(delta3))
    delta4=look_in_tree(hashCode_DWT_query,BST_DWT,DWT_inorder,thresh_DWT)
    #print(len(delta4))
    finish_t=time.process_time() - start
    print(f"look up is done in {finish_t}")
    dct_size=sys.getsizeof(BST_DCT)
    print(f"size of DCT tree is {dct_size}")
    dwt1_size=sys.getsizeof(BST_DWT1)
    print(f"size of DCT tree is {dwt1_size}")
    dwt2_size=sys.getsizeof(BST_DWT2)
    print(f"size of DCT tree is {dwt2_size}")
    dwt_size=sys.getsizeof(BST_DWT)
    print(f"size of DCT tree is {dwt_size}")
    if (len(delta1)>t1 and len(delta2)>t2 and len(delta3)>t3):
        break

  0%|          | 0/1 [00:00<?, ?it/s]

tree building is done in 0.0


IndexError: list index out of range